In [ ]:
!pip install transformers sentence-transformers faiss-cpu torch

In [ ]:

!ngrok config add-authtoken 31CnZgV0BW5STRbU7ASbjoaRird_4kB842X3HP5w1ME8YY4qy


In [ ]:
from transformers import pipeline, AutoTokenizer
import torch

# Use GPU if available
device = 0 if torch.cuda.is_available() else -1



In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

In [ ]:
qa_model = "deepset/roberta-base-squad2"
qa = pipeline("question-answering", model=qa_model, tokenizer=qa_model, device=device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
def chunk_text_by_tokens(text, tokenizer, max_tokens=1024, overlap=128):
    tokens = tokenizer.encode(text)
    chunks = []
    i = 0
    while i < len(tokens):
        chunk = tokenizer.decode(tokens[i:i+max_tokens], skip_special_tokens=True)
        chunks.append(chunk)
        i += max_tokens - overlap
    return chunks

In [ ]:
def summarize_long_text(text):
    chunks = chunk_text_by_tokens(text, tokenizer)
    summaries = []
    for chunk in chunks:
        out = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(out)
    # Merge and summarize again for final concise version
    merged = " ".join(summaries)
    final_summary = summarizer(merged, max_length=200, min_length=60, do_sample=False)[0]['summary_text']
    return final_summary


In [ ]:
def summarize_long_text(text):
    chunks = chunk_text_by_tokens(text, tokenizer)
    summaries = []
    for chunk in chunks:
        token_count = len(tokenizer.encode(chunk))
        # Make max_length smaller than input tokens (e.g., 40% of it)
        max_len = max(30, int(token_count * 0.4))
        min_len = max(10, int(token_count * 0.2))

        out = summarizer(
            chunk,
            max_length=max_len,
            min_length=min_len,
            do_sample=False
        )[0]['summary_text']
        summaries.append(out)

    # Merge and do final summarization
    merged = " ".join(summaries)
    token_count = len(tokenizer.encode(merged))
    max_len = max(50, int(token_count * 0.4))
    min_len = max(20, int(token_count * 0.2))

    final_summary = summarizer(
        merged,
        max_length=max_len,
        min_length=min_len,
        do_sample=False
    )[0]['summary_text']

    return final_summary


In [ ]:
def answer_question(question, text):
    chunks = chunk_text_by_tokens(text, tokenizer)
    best_score = 0
    best_answer = ""
    for chunk in chunks:
        result = qa(question=question, context=chunk)
        if result['score'] > best_score:
            best_score = result['score']
            best_answer = result['answer']
    return best_answer, best_score


# **Testing**

In [ ]:
# Example text (replace with your article)
article = """The James Webb Space Telescope (JWST) is the largest optical telescope in space.
It was launched on December 25, 2021, and is designed primarily to conduct infrared astronomy.
JWST's high sensitivity allows it to view objects too old, distant, or faint for the Hubble Space Telescope."""

# Summarize
print("SUMMARY:\n", summarize_long_text(article))

# Ask question
question = "When was JWST launched?"
answer, score = answer_question(question, article)
print(f"\nQ: {question}\nA: {answer} (score: {score:.3f})")

In [ ]:
article = """
Artificial Intelligence (AI) has been rapidly evolving over the past few decades, moving from a niche research topic to one of the most significant drivers of technological change in the 21st century. Initially confined to rule-based systems and symbolic reasoning, AI has undergone a dramatic transformation, fueled by the advent of machine learning, deep learning, and access to large datasets.

One of the key milestones in AI development was the creation of algorithms capable of learning from data. Unlike traditional software, where programmers explicitly define the rules, machine learning systems identify patterns and relationships in data to make predictions or decisions. This shift has allowed AI to be applied to a wide range of tasks, from image recognition and natural language processing to autonomous vehicles and medical diagnosis.

Deep learning, a subfield of machine learning inspired by the structure and function of the human brain, has been especially impactful. Neural networks with many layers have shown remarkable capabilities in tasks that were once considered impossible for machines. For example, convolutional neural networks (CNNs) have revolutionized computer vision, enabling breakthroughs in facial recognition, object detection, and even art generation.

The availability of vast amounts of data has played a crucial role in this transformation. The explosion of digital content, combined with advances in storage and cloud computing, has made it possible to train AI models on datasets of unprecedented scale. This, in turn, has led to more accurate and robust systems, but it has also raised concerns about privacy, security, and ethical use of data.

Ethics in AI has become a major area of discussion among researchers, policymakers, and the general public. Issues such as algorithmic bias, transparency, accountability, and the potential for job displacement have sparked debates worldwide. Many experts argue that while AI has the potential to bring enormous benefits, it must be developed and deployed responsibly to ensure fairness and prevent harm.

In the business world, AI is being used to optimize operations, enhance customer experiences, and create new products and services. For instance, recommendation systems, powered by AI algorithms, drive much of the engagement on platforms like Netflix, YouTube, and Amazon. In manufacturing, AI-powered predictive maintenance helps reduce downtime and improve efficiency.

Healthcare has also seen transformative effects from AI. Machine learning algorithms can analyze medical images to detect diseases earlier than human doctors, assist in drug discovery by predicting molecular interactions, and even personalize treatment plans based on a patient’s genetic profile. These applications have the potential to save lives and reduce healthcare costs, but they also raise questions about the role of human judgment in medicine.

Autonomous vehicles are another high-profile example of AI in action. Self-driving cars rely on a combination of sensors, cameras, and AI algorithms to navigate complex environments, recognize traffic signs, and avoid obstacles. While significant progress has been made, challenges remain in ensuring safety and dealing with unpredictable real-world scenarios.

Looking ahead, AI is expected to continue advancing at a rapid pace. Emerging areas such as reinforcement learning, generative AI, and neuromorphic computing promise to push the boundaries of what AI systems can achieve. However, these advancements will also bring new challenges, including the need for more energy-efficient models and strategies to mitigate potential misuse.

In conclusion, artificial intelligence has already reshaped many aspects of society, and its influence will only grow in the coming years. By addressing ethical concerns and ensuring responsible development, humanity can harness the power of AI to create a more prosperous, equitable, and sustainable future.
"""

In [ ]:
print("SUMMARY:\n", summarize_long_text(article))

question = "What are some ethical concerns related to AI?"
answer, score = answer_question(question, article)
print(f"\nQ: {question}\nA: {answer} (score: {score:.3f})")

# **Saving**

In [ ]:
from transformers import pipeline


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


summarizer.model.save_pretrained("/content/bart_summarizer")
summarizer.tokenizer.save_pretrained("/content/bart_summarizer")

# ***Deployment***

In [ ]:
%%writefile app.py
# app.py
import streamlit as st
from transformers import pipeline

# Load models once
@st.cache_resource
def load_pipelines():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
    return summarizer, qa_pipeline

summarizer, qa_pipeline = load_pipelines()

# Page settings
st.set_page_config(
    page_title="AI Summarizer & Q&A",
    page_icon="🤖",
    layout="wide",
)

# Title
st.markdown("<h1 style='text-align: center; color: #4CAF50;'> AI Summarizer & Question Answering</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center; font-size: 18px;'>Paste an article, get a concise summary, and ask questions instantly!</p>", unsafe_allow_html=True)
st.markdown("---")

# Layout: Two columns
col1, col2 = st.columns([2, 1])

# Text input
with col1:
    st.markdown("###  Article Input")
    text_input = st.text_area("Enter your article text below:", height=300, placeholder="Paste your text here...")

# Summarization
with col2:
    st.markdown("### Summary")
    if st.button("Summarize", use_container_width=True, type="primary"):
        if text_input.strip():
            summary = summarizer(text_input, max_length=150, min_length=30, do_sample=False)
            st.success(summary[0]['summary_text'])
        else:
            st.warning("Please enter some text to summarize.")

# Question answering section
st.markdown("---")
st.markdown("### ❓ Ask a Question")
question = st.text_input("Enter your question here:", placeholder="e.g., What is the main topic?")

if st.button("Get Answer", use_container_width=True):
    if text_input.strip() and question.strip():
        answer = qa_pipeline(question=question, context=text_input)
        st.info(f"**Answer:** {answer['answer']}")
    else:
        st.warning("Please enter both text and a question.")

# Footer
st.markdown("---")
st.markdown("<p style='text-align: center; font-size: 14px;'>Powered by HuggingFace Transformers ;)</p>", unsafe_allow_html=True)


In [ ]:
from pyngrok import ngrok
import time

# Kill any running Streamlit instances
!pkill streamlit

# Start Streamlit in background
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

# Wait for server to be ready
time.sleep(5)

# Create tunnel
public_url = ngrok.connect(8501)
print(f"Your app is live here: {public_url}")
